### Regression review
- Outcome is real valued

### Common regression metrics
- Root mean squared error (RMSE)
- Mean absolute error (MAE)

### Common regression algorithms
- Linear regression
- Decision trees

### Objective (loss) functions and base learners
- Quantifies how far off a prediction is from the actual result
- Measures the difference between estimated and true values for some collection of data
- Goal: Find the model that yields the minimum value of the loss function